# Validación de Estabilidad en Clustering

La validación de estabilidad evalúa cómo los resultados del clustering cambian cuando se aplica el mismo método a múltiples muestras del conjunto de datos o cuando se cambian los parámetros del algoritmo. Esta es una forma efectiva de medir la robustez de un modelo de clustering.

## Objetivos

- Evaluar la estabilidad de diferentes métodos de clustering (K-means, LDA, Clustering Jerárquico, DBSCAN) aplicados al conjunto de datos `hado_data_clustering.csv`.

- Utilizar métricas como el índice de Rand ajustado para comparar los resultados de diferentes ejecuciones o configuraciones.

## Utilizando LabelEncoder para Convertir Datos Categóricos a Numéricos
Si tenemos columnas con datos categóricos o de texto, podemos utilizar `LabelEncoder` de scikit-learn para convertir estas columnas en valores numéricos. Esto es especialmente útil para algoritmos de aprendizaje automático que requieren entradas numéricas.

In [ ]:
# Load the data
hado_data = pd.read_csv('hado_encoded.csv')

# Show the first few rows of the DataFrame
hado_data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Instanciar el LabelEncoder
le = LabelEncoder()

# Asumiendo que 'columna_categorica' es la columna que queremos transformar
for col in hado_data.select_dtypes(include='object'):
    hado_data[f"´{col}_encoded"] = le.fit_transform(hado_data[col])

# Mostrar las primeras filas para verificar la transformación
hado_data.head()

In [ ]:
hado_encoded = hado_data.select_dtypes(exclude='object')

## Métodos de Clustering a Evaluar

Vamos a evaluar la estabilidad de los siguientes métodos de clustering:

- K-means

- LDA (Análisis Discriminante Lineal)

- Clustering Jerárquico

- DBSCAN

Para cada método, realizaremos múltiples ejecuciones con diferentes configuraciones de parámetros o subconjuntos de datos. Luego, compararemos los resultados utilizando el índice de Rand ajustado.

## Evaluación de Estabilidad para K-means

Para evaluar la estabilidad de K-means, realizaremos lo siguiente:

- Ejecutaremos el algoritmo K-means varias veces con diferentes números de clusters (k).

- Compararemos los resultados de cada ejecución utilizando el índice de Rand ajustado.

In [ ]:
# Importar las bibliotecas necesarias
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# Lista para almacenar los resultados de diferentes ejecuciones de K-means
kmeans_results = []

# Ejecutar K-means con diferentes números de clusters (k=2, k=3, k=4)
for k in [2, 3, 4]:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans_labels = kmeans.fit_predict(hado_encoded)
    kmeans_results.append(kmeans_labels)

# Calcular el índice de Rand ajustado para comparar los resultados
ari_2_3 = adjusted_rand_score(kmeans_results[0], kmeans_results[1])
ari_2_4 = adjusted_rand_score(kmeans_results[0], kmeans_results[2])
ari_3_4 = adjusted_rand_score(kmeans_results[1], kmeans_results[2])

# Mostrar los resultados
print(f'Índice de Rand ajustado entre k=2 y k=3: {ari_2_3}')
print(f'Índice de Rand ajustado entre k=2 y k=4: {ari_2_4}')
print(f'Índice de Rand ajustado entre k=3 y k=4: {ari_3_4}')

## Evaluación de Estabilidad para LDA (Análisis Discriminante Lineal)

Para evaluar la estabilidad de LDA, realizaremos lo siguiente:

- Ejecutaremos el algoritmo LDA varias veces con diferentes números de tópicos.

- Compararemos los resultados de cada ejecución utilizando el índice de Rand ajustado.

In [ ]:
# Importar las bibliotecas necesarias para LDA
from sklearn.decomposition import LatentDirichletAllocation

# Lista para almacenar los resultados de diferentes ejecuciones de LDA
lda_results = []

# Ejecutar LDA con diferentes números de tópicos (n=2, n=3, n=4)
for n in [2, 3, 4]:
    lda = LatentDirichletAllocation(n_components=n, random_state=42)
    lda_labels = lda.fit_transform(hado_encoded)
    lda_results.append(lda_labels.argmax(axis=1))

# Calcular el índice de Rand ajustado para comparar los resultados
ari_lda_2_3 = adjusted_rand_score(lda_results[0], lda_results[1])
ari_lda_2_4 = adjusted_rand_score(lda_results[0], lda_results[2])
ari_lda_3_4 = adjusted_rand_score(lda_results[1], lda_results[2])

# Mostrar los resultados
print(f'Índice de Rand ajustado entre n=2 y n=3: {ari_lda_2_3}')
print(f'Índice de Rand ajustado entre n=2 y n=4: {ari_lda_2_4}')
print(f'Índice de Rand ajustado entre n=3 y n=4: {ari_lda_3_4}')

## Evaluación de Estabilidad para Clustering Jerárquico

Para evaluar la estabilidad del Clustering Jerárquico, realizaremos lo siguiente:

- Ejecutaremos el algoritmo de Clustering Jerárquico varias veces con diferentes métodos de enlace.

- Compararemos los resultados de cada ejecución utilizando el índice de Rand ajustado.

In [ ]:
# Importar las bibliotecas necesarias para Clustering Jerárquico
from scipy.cluster.hierarchy import linkage, fcluster

# Lista para almacenar los resultados de diferentes ejecuciones de Clustering Jerárquico
hierarchical_results = []

# Ejecutar Clustering Jerárquico con diferentes métodos de enlace ('single', 'complete', 'average')
for method in ['single', 'complete', 'average']:
    Z = linkage(hado_encoded, method=method)
    hierarchical_labels = fcluster(Z, t=3, criterion='maxclust')
    hierarchical_results.append(hierarchical_labels)

# Calcular el índice de Rand ajustado para comparar los resultados
ari_hierarchical_single_complete = adjusted_rand_score(hierarchical_results[0], hierarchical_results[1])
ari_hierarchical_single_average = adjusted_rand_score(hierarchical_results[0], hierarchical_results[2])
ari_hierarchical_complete_average = adjusted_rand_score(hierarchical_results[1], hierarchical_results[2])

# Mostrar los resultados
print(f'Índice de Rand ajustado entre enlace simple y enlace completo: {ari_hierarchical_single_complete}')
print(f'Índice de Rand ajustado entre enlace simple y enlace promedio: {ari_hierarchical_single_average}')
print(f'Índice de Rand ajustado entre enlace completo y enlace promedio: {ari_hierarchical_complete_average}')

## Evaluación de Estabilidad para DBSCAN

Para evaluar la estabilidad de DBSCAN, realizaremos lo siguiente:

- Ejecutaremos el algoritmo DBSCAN varias veces con diferentes valores de epsilon y minPts.

- Compararemos los resultados de cada ejecución utilizando el índice de Rand ajustado.

In [ ]:
# Importar las bibliotecas necesarias para DBSCAN
from sklearn.cluster import DBSCAN

# Lista para almacenar los resultados de diferentes ejecuciones de DBSCAN
dbscan_results = []

# Ejecutar DBSCAN con diferentes valores de epsilon y minPts
for eps, minPts in [(0.5, 5), (0.6, 6), (0.7, 7)]:
    dbscan = DBSCAN(eps=eps, min_samples=minPts)
    dbscan_labels = dbscan.fit_predict(hado_encoded)
    dbscan_results.append(dbscan_labels)

# Calcular el índice de Rand ajustado para comparar los resultados
ari_dbscan_1_2 = adjusted_rand_score(dbscan_results[0], dbscan_results[1])
ari_dbscan_1_3 = adjusted_rand_score(dbscan_results[0], dbscan_results[2])
ari_dbscan_2_3 = adjusted_rand_score(dbscan_results[1], dbscan_results[2])

# Mostrar los resultados
print(f'Índice de Rand ajustado entre eps=0.5, minPts=5 y eps=0.6, minPts=6: {ari_dbscan_1_2}')
print(f'Índice de Rand ajustado entre eps=0.5, minPts=5 y eps=0.7, minPts=7: {ari_dbscan_1_3}')
print(f'Índice de Rand ajustado entre eps=0.6, minPts=6 y eps=0.7, minPts=7: {ari_dbscan_2_3}')

## Interpretación de Resultados

### K-means:
- Los resultados entre k=2 y k=3, así como entre k=2 y k=4, tienen un índice de Rand ajustado bajo, lo que indica que los clusters 
son bastante diferentes entre sí.
- Sin embargo, los resultados entre k=3 y k=4 tienen un índice de Rand ajustado más alto, lo que sugiere una mayor estabilidad en la asignación de clusters cuando k varía en ese rango.

### LDA:
- Los resultados muestran una estabilidad moderada cuando se varía el número de temas (n). Los índices de Rand ajustados son más altos que en K-means pero aún no son muy cercanos a 1.

### Clustering Jerárquico:
- Los resultados son extremadamente estables entre diferentes tipos de enlace (simple, completo, promedio), con índices de Rand ajustados cercanos a 1.

### DBSCAN:
- Los resultados son completamente estables cuando se varían los parámetros eps y minPts, con índices de Rand ajustados de 1.

## Conclusiones:
1. **K-means** muestra la menor estabilidad entre las diferentes configuraciones.
2. **LDA** muestra una estabilidad moderada.
3. **Clustering Jerárquico** y **DBSCAN** muestran una alta estabilidad, lo que sugiere que son más robustos a las variaciones en sus parámetros.

Estos resultados pueden ser útiles para entender qué algoritmo es más adecuado para tu caso de uso específico. Si la estabilidad es una preocupación, podrías considerar usar Clustering Jerárquico o DBSCAN. Sin embargo, si estás más interesado en explorar diferentes estructuras de datos, K-means y LDA podrían ser más apropiados.

## Notas finales

En este notebook, hemos evaluado la estabilidad de varios métodos de clustering utilizando el índice de Rand ajustado como métrica de comparación. Los resultados nos ayudan a entender qué tan robustos son estos métodos bajo diferentes configuraciones o subconjuntos de datos.

Es crucial tener en cuenta que la estabilidad no es el único factor a considerar al elegir un método de clustering. También es importante considerar la interpretabilidad, la eficiencia computacional y cómo se ajustan los resultados a los objetivos del análisis.